In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
from torchvision import models, datasets
import torchvision.transforms as transforms
from tqdm.notebook import trange, tqdm
import matplotlib.pyplot as plt
from scipy.io import loadmat
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, random_split
import torchvision.models as models

# Hyperparameters
N_EPOCHS = 0
LEARNING_RATE = 0.0001
TEST_SIZE=0.20
BATCH_SIZE=128

# Transformations for our dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 128x128
    transforms.ToTensor(),          # Convert PIL image to PyTorch tensor
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize to range [-1, 1] for better training stability
])

# Load the dataset
root_dir = 'data_zadanie2/'  # Modify this to the path where your image folders (cat, chicken, etc.) are located
full_dataset = datasets.ImageFolder(root=root_dir, transform=transform)

# Splitting the dataset
train_size = int((1 - TEST_SIZE) * len(full_dataset))
test_size = len(full_dataset) - train_size

train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

# Dataloaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
# Load the pre-trained AlexNet model
def get_alex_model():
    alexnet = models.alexnet(pretrained=True)
    num_layers = 0

    for param in alexnet.parameters():
        param.requires_grad = False
        num_layers = num_layers + 1

    alexnet.classifier[6] = nn.Linear(4096, 8)  
    alexnet.eval()
    input_tensor = torch.randn(1, 3, 224, 224)  # Example random input
    alexnet(input_tensor)
    num_layers

    return alexnet

def get_vgg16_model():
    vgg16 = models.vgg16(pretrained=True)
    num_layers = 0

    for param in vgg16.features.parameters():
        param.requires_grad = False
        num_layers = num_layers + 1

    num_features = vgg16.classifier[6].in_features
    vgg16.classifier[6] = nn.Linear(num_features, 8)  # Modify the output layer for your specific task
    vgg16.eval()
    input_tensor = torch.randn(1, 3, 224, 224)  # Example random input
    vgg16(input_tensor)
    
    return vgg16

def get_resnet18_model():
    resnet18 = models.resnet18(pretrained=True)
    num_layers = 0

    for param in resnet18.parameters():
        param.requires_grad = False
        num_layers = num_layers + 1

    num_features = resnet18.fc.in_features
    resnet18.fc = nn.Linear(num_features, 8)  # Modify the output layer for your specific task
    resnet18.eval()
    input_tensor = torch.randn(1, 3, 224, 224)  # Example random input
    resnet18(input_tensor)
    
    return resnet18


def get_mobilenet_model():
    mobilenet = models.mobilenet_v2(pretrained=True)
    num_layers = 0

    for param in mobilenet.parameters():
        param.requires_grad = False
        num_layers = num_layers + 1

    num_features = mobilenet.classifier[1].in_features
    mobilenet.classifier[1] = nn.Linear(num_features, 8)  # Modify the output layer for your specific task
    mobilenet.eval()
    input_tensor = torch.randn(1, 3, 224, 224)  # Example random input
    mobilenet(input_tensor)
    
    return mobilenet

step = 16/4
step = 26/4
step = 62/4
step = 158/4

In [ ]:
def evaluate_model(model, dataloader):
    """
    Evaluate the given model using the specified dataloader.

    Args:
    - model: Trained PyTorch model
    - dataloader: DataLoader for the evaluation data

    Returns:
    - Accuracy of the model on the evaluation data
    """
    correct = 0
    total = 0
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()  # Set model to evaluation mode

    with torch.no_grad():  # No need to track gradients during evaluation
        for inputs, labels in tqdm(test_loader, desc="Evaluation", leave=False):
            inputs, labels = inputs.to(device).float(), labels.to(device)  # Move inputs and labels to the device and ensure float32 type
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy

In [ ]:
# Training
def train_model(model, optimizer, criterion, dataloader, epochs, unfreez, save_name):
    """
    Train a given model using the specified optimizer and criterion.

    Args:
    - model: PyTorch model to be trained
    - optimizer: Optimizer for updating model parameters
    - criterion: Loss function
    - dataloader: DataLoader for the training data
    - epochs: Number of epochs (default=10)

    Returns:
    - list of losses per epoch
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.train()  # Set the model to training mode
    loss_list = []  # To store the loss per epoch
    accuracy_list= []
    freeze_first_n_layers = 0 
    
    # Loop over epochs
    for epoch in trange(epochs, desc="Epochs"):
        # ... [rest of the code remains unchanged]
        epoch_loss = 0  # Initialize epoch loss
        model.zero_grad()
        count = 0

        if epoch < 3:
            freeze_first_n_layers = freeze_first_n_layers - step
        elif epoch < 6:
            freeze_first_n_layers = freeze_first_n_layers - 2*step
        elif epoch < 9:
            freeze_first_n_layers = freeze_first_n_layers - 3*step
        elif epoch < 12:
           freeze_first_n_layers = freeze_first_n_layers - 4*step

        if unfreez:
            # unfreeze backbone layers
            for param in model.children():
                count +=1
                if count > freeze_first_n_layers: 
                    param.requires_grad = True     

        for inputs, labels in tqdm(train_loader, desc="Batches", leave=False):
            inputs, labels = inputs.to(device).float(), labels.to(device)
            optimizer.zero_grad()  # Clear gradients
            outputs = model(inputs)  # Forward pass
            loss = criterion(outputs, labels)  # Calculate loss
            loss.backward()  # Backward pass
            optimizer.step()  # Update weights

            epoch_loss += loss.item()
        accuracy=evaluate_model(model, DataLoader(test_dataset, batch_size=BATCH_SIZE))
        accuracy_list.append(accuracy)
        avg_loss = epoch_loss / len(dataloader)
        loss_list.append(avg_loss)
        print("Epoch {}, Loss {:.6f}, Accuracy {:.2f}% ".format(epoch, avg_loss, accuracy))

    torch.save(model.state_dict(), 'modely_zadanie_2/' + save_name + ".pth")   
    model.eval()
    return loss_list,accuracy_list

In [ ]:
if torch.cuda.is_available():
    print("cuda")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

criterion = nn.CrossEntropyLoss()

# Training the CNN model
cnn_model1 = get_mobilenet_model()
cnn_optimizer = torch.optim.Adam(cnn_model1.parameters(), lr=LEARNING_RATE)
cnn_model1.to(device)
cnn_losses1,cnn_accuracy_list1 = train_model(cnn_model1, cnn_optimizer, criterion, DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True), epochs=N_EPOCHS, unfreez=True, save_name='mobilenet_model_1')
# Evaluating the CNN model
cnn_accuracy = evaluate_model(cnn_model1, DataLoader(test_dataset, batch_size=BATCH_SIZE))
print(f"CNN Model Accuracy: {cnn_accuracy:.2f}%")

cnn_model2 = get_mobilenet_model()
cnn_optimizer = torch.optim.Adam(cnn_model2.parameters(), lr=LEARNING_RATE)
cnn_model2.to(device)
cnn_losses2,cnn_accuracy_list2 = train_model(cnn_model2, cnn_optimizer, criterion, DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True), epochs=N_EPOCHS, unfreez=False,save_name='mobilenet_model_2')
# Evaluating the CNN model
cnn_accuracy = evaluate_model(cnn_model2, DataLoader(test_dataset, batch_size=BATCH_SIZE))
print(f"CNN Model Accuracy: {cnn_accuracy:.2f}%")

# Augmentation
# Define a custom dataset class for rotated images
import random
class RotatedImageDataset(Dataset):
    def __init__(self, original_dataset, rotate_prob=1):
        self.original_dataset = original_dataset
        self.rotate_prob = rotate_prob

    def __len__(self):
        return len(self.original_dataset)

    def __getitem__(self, idx):
        image, label = self.original_dataset[idx]

        # Randomly decide whether to rotate the image or not
        should_rotate = random.random() < self.rotate_prob

        if should_rotate:
            # Rotate the image by 90 degrees
            image = transforms.functional.rotate(image, 90)

        return image, label

# Hyperparameters
TEST_SIZE=0.40

# Transformations for our dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)), 
    transforms.ToTensor(),          
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize to range [-1, 1] for better training stability
])

# Load the dataset
root_dir = 'data_zadanie2/'  # Modify this to the path where your image folders (cat, chicken, etc.) are located
original_dataset = datasets.ImageFolder(root=root_dir, transform=transform)

augment_data_size = int((1 - 0.6) * len(original_dataset))
unused_size = len(original_dataset) - augment_data_size
augment_data, unused = random_split(original_dataset, [augment_data_size, unused_size])
rotated_dataset = RotatedImageDataset(augment_data, rotate_prob=1)

train_size = int((1 - TEST_SIZE) * len(original_dataset))
test_size = len(original_dataset) - train_size
train_dataset, test_dataset = random_split(original_dataset, [train_size, test_size])
train_dataset = torch.utils.data.ConcatDataset([train_dataset, rotated_dataset])

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

cnn_model3 = get_mobilenet_model()
cnn_optimizer = torch.optim.Adam(cnn_model3.parameters(), lr=LEARNING_RATE)
cnn_model3.to(device)
cnn_losses3,cnn_accuracy_list3 = train_model(cnn_model3, cnn_optimizer, criterion, DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True), epochs=N_EPOCHS, unfreez=True,save_name='mobilenet_model_3')
# Evaluating the CNN model
cnn_accuracy = evaluate_model(cnn_model3, DataLoader(test_dataset, batch_size=BATCH_SIZE))
print(f"CNN Model Accuracy: {cnn_accuracy:.2f}%")




In [ ]:
import matplotlib.pyplot as plt

def plot_accuracy(accuracy_list1,accuracy_list2,accuracy_list3, model_name):

    plt.figure(figsize=(12, 6))
    plt.plot(accuracy_list1, '-o', label=f'{model_name} Accuracy - Attempt 1')
    plt.plot(accuracy_list2, '-o', label=f'{model_name} Accuracy - Attempt 2')
    plt.plot(accuracy_list3, '-o', label=f'{model_name} Accuracy - Attempt 3')
    plt.title(f'Accuracy Evolution for {model_name}')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.legend()
    plt.grid(True, which='both', linestyle='--', linewidth=0.5)
    plt.tight_layout()
    plt.show()



In [ ]:
import matplotlib.pyplot as plt

def plot_losses(cnn_losses1,cnn_losses2,cnn_losses3, model_name):

    plt.figure(figsize=(12, 6))
    plt.plot(cnn_losses1, '-o', label=f'{model_name} Losses  Attempt 1')
    plt.plot(cnn_losses2, '-o', label=f'{model_name} Losses  Attempt 2')
    plt.plot(cnn_losses3, '-o', label=f'{model_name} Losses  Attempt 3')
    plt.title(f'Training Loss evolution for {model_name}')
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend()
    plt.grid(True)
    plt.show()



In [ ]:

def plot_filters(layer):
    # Extract filter weights from the layer and convert to numpy for visualization.
    filters = layer.weight.data.cpu().numpy()
    
    # Calculate the number of rows and columns for the grid.
    num_filters = filters.shape[0]
    num_rows = num_columns = int(np.ceil(np.sqrt(num_filters)))
    
    # Initialize a figure for displaying the filter images.
    fig, axes = plt.subplots(num_rows, num_columns, figsize=(10, 10))
    
    # Loop through each filter in the layer and display it in the grid.
    for i in range(num_filters):
        row, col = i // num_columns, i % num_columns
        ax = axes[row, col]
        ax.imshow(filters[i][0], cmap='gray')
        ax.axis('off')
    
    # Remove any empty subplots in the grid.
    for i in range(num_filters, num_rows * num_columns):
        fig.delaxes(axes.flatten()[i])
    
    plt.show()



In [ ]:
def plot_filters(layer):
    # Extract filter weights from the layer and convert to numpy for visualization.
    filters = layer.weight.data.cpu().numpy()
    
    # Calculate the number of rows and columns for the grid.
    num_filters = filters.shape[0]
    num_rows = num_columns = int(np.ceil(np.sqrt(num_filters)))
    
    # Initialize a figure for displaying the filter images.
    fig, axes = plt.subplots(num_rows, num_columns, figsize=(10, 10)
                            , gridspec_kw={'wspace': 0.4, 'hspace': 0.4})
    
    # Loop through each filter in the layer and display it in the grid.
    for i in range(num_filters):
        row, col = i // num_columns, i % num_columns
        ax = axes[row, col]
        ax.imshow(filters[i][0], cmap='gray')
        ax.axis('off')
        ax.set_title(f'Filter {i}')
    
    # Remove any empty subplots in the grid.
    for i in range(num_filters, num_rows * num_columns):
        fig.delaxes(axes.flatten()[i])
    
    plt.show()


In [ ]:
def visualize_feature_maps(model, input_image):
    # Making sure model is in eval mode
    model.eval()
    
    # Running the image through the first conv layer
    first_conv_layer = model.features[0]
    second_conv_layer = model.features[2] 

    feature_maps1 = first_conv_layer(input_image.unsqueeze(0).float().to(device))
    feature_maps2 = second_conv_layer(feature_maps1)
    # If you want to visualize after the second conv layer
    # feature_maps2 = model.conv2(F.relu(feature_maps1))
    
    n_feature_maps1 = feature_maps1.shape[1]
    n_feature_maps2 = feature_maps2.shape[1]
    
    # Plotting each feature map
    for i in range(n_feature_maps1):
        plt.subplot(8, 8, i+1)  # Assuming a max of 64 feature maps for display
        plt.imshow(feature_maps1[0][i].cpu().detach().numpy(), cmap='gray')
        plt.axis('off')
    plt.show()

    for i in range(n_feature_maps2):
        plt.subplot(8, 8, i+1)  # Assuming a max of 64 feature maps for display
        plt.imshow(feature_maps2[0][i].cpu().detach().numpy(), cmap='gray')
        plt.axis('off')
    plt.show()

# Load the model checkpoint
checkpoint = torch.load('modely_zadanie_2/VGG16_model_2.pth')
model = get_vgg16_model()
model.load_state_dict(checkpoint)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)   

# Select a random image from the dataset
image, _ = test_dataset[1]
plt.imshow(image[0], cmap='gray')  # Displaying the selected image
plt.show()

# Displaying feature maps for the selected image
# visualize_feature_maps(model, image)
plot_filters(model.features[0])
plot_filters(model.features[2])

In [ ]:
plot_accuracy(cnn_accuracy_list1, cnn_accuracy_list2, cnn_accuracy_list3, "MobileNet")
plot_losses(cnn_losses1, cnn_losses2, cnn_losses3, "MobileNet")

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

# Load the model checkpoint
checkpoint = torch.load('modely_zadanie_2/VGG16_model_3.pth')
model = get_vgg16_model()
model.load_state_dict(checkpoint)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def get_all_preds_and_labels(model, dataloader):
    # Empty tensors to store all predictions and true labels.
    all_preds = torch.tensor([]).to(device)
    all_labels = torch.tensor([]).to(device)
    
    # Loop through the data in the dataloader.
    for inputs, labels in dataloader:
        # Convert inputs to float32
        inputs = inputs.float()
        # Move the data to the device (GPU/CPU).
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Forward pass to get predictions.
        outputs = model(inputs)
        
        # Get the index (class label) with the maximum score as the prediction.
        _, preds = torch.max(outputs, 1)
        
        # Append the batch's predictions and labels to the accumulating tensors.
        all_preds = torch.cat((all_preds, preds), dim=0)
        all_labels = torch.cat((all_labels, labels), dim=0)
    
    # Return all predictions and true labels.
    return all_preds, all_labels

import numpy as np
from sklearn import metrics

def plot_confusion_matrix(all_preds, all_labels):
    # Compute the confusion matrix using sklearn's function.
    cm = metrics.confusion_matrix(all_labels.cpu(), all_preds.cpu())
    
    # Define the labels you're using (in this case, it's A-J).
    labels = ["cat","chicken","cow","dog","elephant","horse","sheep","squirrel"]
    
    # Create the ConfusionMatrixDisplay object.
    cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
    
    # Plot the confusion matrix.
    cm_display.plot(cmap='Blues')
    plt.show()

# After training your CNN model
cnn_model = model.float()
all_preds, all_labels = get_all_preds_and_labels(cnn_model, DataLoader(test_dataset, batch_size=64))
plot_confusion_matrix(all_preds, all_labels)
